In [2]:
#libs for mysql connect
from sqlalchemy import create_engine
import credentials
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Date/time functionality
import datetime
import time
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
import shutil
import os
import glob

Cleaning tables

In [2]:
dtypes={'ROUTEID':'category', 'Beginning_stop':'category','Ending_stop':'category', 'Weight':'float64' }
df_ssp_table = pd.read_csv('stop_point_pair_vs_routeid.csv')

In [3]:
df_ssp_table.head(2)

,ROUTEID,Beginning_stop,Ending_stop,Weight
0,18_3,4359,4360,0.006136
1,18_3,4359,4361,0.006136


In [3]:
home_path=os.path.normpath(os.getcwd() + os.sep + os.pardir)
df_tt_table=pd.read_csv(home_path+'/Lorna/timetables/timetables.csv')

In [5]:
df_tt_table.head(2)

,ROUTEID,DAY_OF_WEEK,STOPPOINTID,PLANNEDTIME_DEP_R_M5,TRIPS_TIME_PROPORTION_v2,PLANNEDTIME_DEP
0,41B_52,Tuesday,1171,06:30,0.029360,23455
1,41B_52,Tuesday,1171,11:35,0.061684,41784


In [4]:
df_ft_table=pd.read_csv(home_path+'/Lorna/FeatureSelection/feature_selection_cor01.csv')

In [5]:
df_ft_table.head(2)

,PLANNEDTIME_DEP,ROUTEID,TOTAL_JOURNEY_TIME,rain,temp,dew_pt_temp(C),humidity(%),sea_lvl_pressure(hPa),sin_hour_of_day,cos_hour_of_day,friday,monday,saturday,sunday,thursday,tuesday,wednesday
0,1,18_3,1,0,0,0,0,0,1,1,1,0,1,1,1,1,1
1,1,67_6,1,0,1,0,1,0,1,1,1,0,1,1,1,1,1


In [6]:
 ROUTEID_drop_df= pd.read_csv('ML_route_bel_tres_R2_cor01_df.csv')

In [7]:
 ROUTEID_drop_df.head(2)

,ROUTEID,R2
0,111_9,-0.026251
1,13_73,0.041089


In [8]:
ROUTEID_drop_list= ROUTEID_drop_df['ROUTEID'].values
ROUTEID_drop_list

array(['111_9', '13_73', '17A_13', '185_51', '185_52', '185_53', '185_55',
       '185_57', '185_59', '185_60', '25D_275', '25D_277', '27B_40',
       '33B_58', '33X_46', '38A_33', '40D_101', '40E_91', '44B_61',
       '47_135', '53_23', '68_79', '68_85', '7B_91', '83_25', '84A_3',
       '84X_57'], dtype=object)

In [9]:
print("Before dropping df_ssp_table ", df_ssp_table.shape[0], " df_tt_table ", df_tt_table.shape[0] )
for ROUTE_ID in ROUTEID_drop_list:
    df_ssp_table.drop(df_ssp_table[df_ssp_table.ROUTEID == ROUTE_ID].index, inplace=True)
    df_tt_table.drop(df_tt_table[df_tt_table.ROUTEID == ROUTE_ID].index, inplace=True)
print("After dropping df_ssp_table ", df_ssp_table.shape[0], " df_tt_table ", df_tt_table.shape[0] )

Before dropping df_ssp_table  483218  df_tt_table  2307034
After dropping df_ssp_table  455817  df_tt_table  2263084


Writing table to mysql db

In [6]:
def get_engine():
    '''Used to create the engine, selects production or test database depending on requirement'''
    db_name = credentials.db_database 
    uri = credentials.db_uri
    user = credentials.db_user
    password = credentials.db_password
    port = credentials.db_port
    return create_engine(f"mysql+pymysql://{user}:{password}@{uri}:{port}/{db_name}",echo=True)

In [7]:
def add_tables(df_tables,table_names):
    engine = get_engine()
    for i in range(len(df_tables)):
        df_tables[i].to_sql(table_names[i], con=engine)
        engine.execute("SELECT * FROM "+table_names[i]).fetchall()

In [13]:
add_tables([df_ssp_table,df_tt_table],['STOPPOINTID_pairs_vs_ROUTEID','timetables'])

2022-07-15 01:39:48,393 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-07-15 01:39:48,394 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:39:48,569 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-07-15 01:39:48,570 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:39:48,657 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-07-15 01:39:48,658 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:39:48,920 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-07-15 01:39:48,921 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {'table_schema': 'static_tables', 'table_name': 'STOPPOINTID_pairs_vs_ROUTEID'}
2022-07-15 01:39:49,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-15 01:39:49,165 INFO sqlalchemy.engine.Engine 
CREATE TABLE `STOPPOINTID_pairs_vs_ROUTEID` (
	`index` BIGINT, 
	`ROUTEID` TEXT, 
	`Beginning_stop` BIGINT, 
	`End

In [8]:
add_tables([df_ft_table],['feature_selection'])

2022-07-15 14:16:38,797 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-07-15 14:16:38,799 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 14:16:38,970 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-07-15 14:16:38,971 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 14:16:39,056 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-07-15 14:16:39,057 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 14:16:39,314 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-07-15 14:16:39,315 INFO sqlalchemy.engine.Engine [generated in 0.00095s] {'table_schema': 'static_tables', 'table_name': 'feature_selection'}
2022-07-15 14:16:39,496 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-15 14:16:39,498 INFO sqlalchemy.engine.Engine 
CREATE TABLE feature_selection (
	`index` BIGINT, 
	`PLANNEDTIME_DEP` BIGINT, 
	`ROUTEID` TEXT, 
	`TOTAL_JOURNEY_TIME` BIGINT

Check if entered data at sql is matching dataframe

In [14]:
df_ssp_table_mysql= pd.read_sql("select * from STOPPOINTID_pairs_vs_ROUTEID", get_engine());

2022-07-15 01:43:32,628 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-07-15 01:43:32,629 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:43:32,805 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-07-15 01:43:32,807 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:43:32,895 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-07-15 01:43:32,896 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:43:33,158 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-07-15 01:43:33,159 INFO sqlalchemy.engine.Engine [generated in 0.00153s] {'table_schema': 'static_tables', 'table_name': 'select * from STOPPOINTID_pairs_vs_ROUTEID'}
2022-07-15 01:43:33,389 INFO sqlalchemy.engine.Engine select * from STOPPOINTID_pairs_vs_ROUTEID
2022-07-15 01:43:33,392 INFO sqlalchemy.engine.Engine [raw sql] {}


In [15]:
df_ssp_table_mysql.shape

(455817, 5)

In [18]:
ssp_missing_row_count=df_ssp_table.shape[0]-df_ssp_table_mysql.shape[0]
ssp_missing_row_count

0

In [19]:
df_ssp_table_mysql.to_csv('STOPPOINTID_pairs_vs_ROUTEID.csv',index=False)

In [20]:
df_tt_table_mysql= pd.read_sql("select * from timetables", get_engine());

2022-07-15 01:45:01,382 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-07-15 01:45:01,384 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:45:01,556 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-07-15 01:45:01,557 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:45:01,644 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-07-15 01:45:01,645 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 01:45:01,901 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-07-15 01:45:01,903 INFO sqlalchemy.engine.Engine [generated in 0.00147s] {'table_schema': 'static_tables', 'table_name': 'select * from timetables'}
2022-07-15 01:45:02,092 INFO sqlalchemy.engine.Engine select * from timetables
2022-07-15 01:45:02,093 INFO sqlalchemy.engine.Engine [raw sql] {}


In [21]:
df_tt_table_mysql.shape

(2263084, 7)

In [22]:
tt_missing_row_count=df_tt_table.shape[0]-df_tt_table_mysql.shape[0]
tt_missing_row_count

0

In [23]:
df_tt_table_mysql.to_csv(home_path+'/Lorna/timetables_V2/timetables.csv',index=False)

In [9]:
#add_tables([df_ft_table],['feature_selection'])
df_ft_table_mysql= pd.read_sql("select * from feature_selection", get_engine());

2022-07-15 14:18:56,802 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-07-15 14:18:56,804 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 14:18:56,980 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-07-15 14:18:56,982 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 14:18:57,069 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-07-15 14:18:57,070 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-15 14:18:57,332 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-07-15 14:18:57,333 INFO sqlalchemy.engine.Engine [generated in 0.00148s] {'table_schema': 'static_tables', 'table_name': 'select * from feature_selection'}
2022-07-15 14:18:57,524 INFO sqlalchemy.engine.Engine select * from feature_selection
2022-07-15 14:18:57,525 INFO sqlalchemy.engine.Engine [raw sql] {}


In [10]:
df_ft_table_mysql.shape

(278, 18)

In [12]:
ft_missing_row_count=df_ft_table.shape[0]-df_ft_table_mysql.shape[0]
ft_missing_row_count

0